In [12]:
import requests
import json
from datetime import datetime, timedelta

# Create timestamps
now = datetime.now()
first_event_time = now - timedelta(minutes=5)  # 5 minutes ago
last_event_time = now

# Build test incident payload
test_incident = {
    "id": f"TEST_{int(now.timestamp())}",
    "timestamp": now.isoformat(),
    "description": json.dumps({
        "rule_id": "9999",
        "rule_description": " Davis",
        "rule_level": 10,
        "total_events": 2,
        "agent_names": ["Blue Iris"],
        "first_event_timestamp": first_event_time.isoformat(),
        "last_event_timestamp": last_event_time.isoformat(),
        "sample_event": {
            "rule": {
                "id": "834613",
                "description": "Critical Camera Alert",
                "level": 10
            },
            "agent": {
                "name": "Blue Iris",
                "id": "001"
            },
            "timestamp": first_event_time.isoformat(),
            "data": {
                "service": "nginx",
                "status": "stopped",
                "exit_code": 1
            }
        },
        "csv_path": "/incidents/test/events.csv"
    }),
    "acknowledged": False,
    "escalated": False
}

# Send POST request using requests module
response = requests.post(
    url='http://0.0.0.0:8000/incidents/',
    headers={'Content-Type': 'application/json'},
    json=test_incident
)

if response.status_code != 200:
    print('Error:', response.status_code, response.text)
else:
    print('Response:', response.json())

Response: {'message': 'Incident created', 'id': 'TEST_1732203209'}


In [1]:
#!/usr/bin/env python3
from auth import register_user, SessionLocal, Base, engine
from fastapi import HTTPException
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def init_db():
    # Create tables
    Base.metadata.create_all(bind=engine)
    
    success = True
    try:
        # Create admin user
        admin_user = register_user("admin", "admin", role="admin")
        logger.info("Admin user created successfully")
        
        # Create analyst user
        analyst_user = register_user("analyst", "analyst", role="analyst")
        logger.info("Analyst user created successfully")
        
    except HTTPException as e:
        if e.detail == "Username already registered":
            logger.info("User already exists")
        else:
            logger.error(f"Error creating user: {e.detail}")
        success = False
    except Exception as e:
        logger.error(f"Unexpected error: {str(e)}")
        success = False
    
    return success

if __name__ == "__main__":
    init_db()

INFO:__main__:Admin user created successfully
INFO:__main__:Analyst user created successfully


In [2]:
#!/usr/bin/env python3
from auth import SessionLocal, User
import logging
from sqlalchemy import inspect

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def print_all_users():
    db = SessionLocal()
    try:
        # Get all columns from User model
        inspector = inspect(User)
        columns = [c.key for c in inspector.mapper.column_attrs]
        
        # Query all users
        users = db.query(User).all()
        
        print("\nAll users in database:")
        for user in users:
            print("-" * 60)
            # Print all columns for each user
            for col in columns:
                value = getattr(user, col)
                print(f"{col}: {value}")
        print("-" * 60)
            
    except Exception as e:
        logger.error(f"Error querying database: {str(e)}")
    finally:
        db.close()

if __name__ == "__main__":
    print_all_users()


All users in database:
------------------------------------------------------------
id: 1
username: admin
password_hash: $2b$12$ap0hKShD7yOCWdy0QPunhuzUU0k3j4thHe9TvjLQfKGiEwEtsdbTy
role: admin
------------------------------------------------------------
id: 2
username: analyst
password_hash: $2b$12$5VBOr4ayHb6hsxSkzTR5BugtExKpTGaWy3ZJ7PWnDztDm2jwMB/BC
role: analyst
------------------------------------------------------------


In [5]:
enabled = "False"
if enabled == "True":
    print("Enabled")

True
